# Flujo estándar

Cada vez que se desea trabajar con las señales de los sismómetros, se debe aplicar una serie de operaciones. En procesamiento digital de señales, esto se conoce como pre-procesamiento. Tiene como fin de ajustar las señales a los requerimientos del sistema, separar la información relevante del resto, disminuir el ruído, entre otras. 

Lo pasos básicos son:

- Ajustar el `sample rate` de todos los `traces` al mismo valor. En general es útil si se van a aplicar operaciones entre ellos, por ejemplo, calcular una cross-correlación. 
- Solucionar los *huecos* con operaciones `.merge()`.
- Ajustar la longitud de la señal con `.trim()`.
- Remover la línea de tendencia con `.detrend()`.
- Remover la respuesta del instrumento con `.remove_response()`. En Obspy, esta función puede tambien remover la media. 
- Filtrar la señal a la banda deseada utilizando `.filter()`. 

El siguiente código aplica los pasos necesarios del flujo a la señal del sismómetro de Jacó:

In [ ]:
import obspy 

# Carga la señal a memoria
jaco_wave = obspy.read('datos/tortugero-sismo_jaco.mseed')
jaco_wave.merge()

# Remueve la línea de tendencia
jaco_wave.detrend(type='linear')

# Carga el archivo con la respuesta del instrumento
resp = obspy.read_inventory('datos/tc_stations.xml')
# Remueve la respuesta del instrumento. Retorna el DESPlazamiento
jaco_wave.remove_response(inventory=resp, output='DISP', water_level=0)

# aplica un filtro paso-bajos a la señal, con frecuencia de corte 0.20 Hz
jaco_wave.filter('lowpass', freq=0.1)

In [ ]:
jaco_wave.plot() 

### Ejercicio 1

Escriba una functión llamda `pre_process` que tome un objeto `Trace` y un objeto `Inventory` y aplique el pre-procesamiento típico. Añada un argumento `key-value` para la frecuencia de corte del filtro paso bajo. Utilice la función para procesar de nuevo la señal del sismómetro de Jacó. 

# Análisis en frecuencia

Analizar las señales en el dominio de la frecuencia es una actividad muy común. Obspy incluye el método `.spectrogram()`, que grafica el espectrograma de la señal. Esto es, una representación en tiempo y frecuencia de la señal. Puede encontrar la documentación de la función [aquí](https://docs.obspy.org/packages/autogen/obspy.imaging.spectrogram.spectrogram.html#obspy.imaging.spectrogram.spectrogram).

In [ ]:
# Preprocesar la señal con un filtro poco discriminante

jaco_wave = obspy.read('datos/tortugero-sismo_jaco.mseed')
resp = obspy.read_inventory('datos/tc_stations.xml')

jaco_wave = pre_process(jaco_wave, resp, freq=25)

jaco_wave.spectrogram()

La función `.spectrogram()` es útil para darse una idea rápida de la señal. Sin embargo, si queremos aplicar nuestros propios análisis, debemos utilizar Numpy junto con Matplotlib. Por ejemplo, podemos calcular la Tansformada de Fourier del sismo y observar con más detalle como se distribuye en la frecuencia:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

jaco_fft = np.fft.rfft(jaco_wave[0].data)
print(jaco_fft)
freq = np.array([i * jaco_wave[0].stats['sampling_rate'] / len(jaco_fft) for i in range(len(jaco_fft))])
plt.semilogx(freq, np.absolute(jaco_fft))
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Potencia espectral')

plt.show()

¡Aparecen cosas interesantes! Por ejemplo:

- La mayor parte de la energía de la señal está en las frecuencias bajas.
- La señal parece estar compuesta de armónicos. 
- La razón señal a ruido empeora al aumentar la frecuencia.


# Transformada Wavelet continua

El espectrograma nos permite visualizar como cambia el espectro de la señal en el tiempo. En ese sentido, la transformada wavelet nos permite hacer lo mismo, sin embargo, esta tiene mejor resolución espacial y temporal. 
El código a continuación calcula la transformada wavelet del sismo de Jacó. 

In [ ]:
import obspy
from scipy import signal
import matplotlib.pyplot as plt 
import numpy as np 

jaco_wave = obspy.read('datos/tortugero-sismo_jaco.mseed')
resp = obspy.read_inventory('datos/tc_stations.xml')

jaco_wave = pre_process(jaco_wave, resp, freq=25) 
plt.plot(np.linspace(0,660, len(jaco_wave[0].data)), jaco_wave[0].data)
plt.xlabel('Tiempo (s)')
plt.ylabel('Desplazamiento (m)')
plt.show()

niveles = np.arange(1,100)
jaco_cwt = signal.cwt(jaco_wave[0].data, signal.ricker, niveles)
plt.imshow(jaco_cwt, aspect='auto', extent=[0,660,99,0], cmap='seismic', vmax=abs(jaco_cwt).max(), vmin=-abs(jaco_cwt).max())
plt.xlabel('Tiempo (s)')
plt.ylabel('Nivel (proporcional a la frecuencia)')
plt.show()

# Soluciones

### Ejercicio 1

In [ ]:
def pre_process(signal, inventario, freq=0.1):
    signal.merge()
    signal.detrend(type='linear')
    signal.remove_response(inventory=inventario, output='DISP', water_level=0)
    signal.filter('lowpass', freq=freq)
    return signal